In [1]:
import numpy as np
import pandas as pd

In [23]:
X_train = pd.read_csv("../Data/train.csv")
df_macro = pd.read_csv("../Data/macro.csv")

In [24]:
y_train = X_train[["price_doc"]]
X_train = X_train.drop("price_doc", axis = 1)

In [37]:
X_train_sample = X_train[["full_sq", "life_sq","floor"]].fillna(0).sample(5)
y_train_sample = y_train.loc[X_train_sample.index]

## Neural Network

In [117]:
class NeuralNetwork():
    def fit(self, X, y, n_hidden, nodes, activations):
        self._X = X.values
        self._y = y.values
        self._n_hidden = n_hidden
        self._nodes = nodes
        self._weights = self._generate_weights()
        self._biases = self._generate_bias()
        self._activations = activations
        
        self._train()
        
    def _activation(self, data, activation = "relu"):
        if activation == "relu":
            def relu(data):
                return np.array([max(0,i) for i in data]).reshape(data.shape)
            return np.apply_along_axis(relu, 1, data)
        if activation == "sigmoid":
            def sigmoid(data):
                return (1/(1 + np.exp(-data))).reshape(data.shape)
            return np.apply_along_axis(sigmoid, 1, data)
        
    def _loss_function(self, ypred, loss = "mse"):
        if loss == "mse":
            return ((y - ypred) ** 2).mean()
    
    def _generate_weights(self):
        hidden_weights = []
        nodes = self._nodes
        for idx in range(1,len(nodes)):
            hidden_weights.append(0.01 * np.random.randn(nodes[idx -1], nodes[idx]))
        return np.array(hidden_weights)
    
    def _generate_bias(self):
        hidden_layers = []
        nodes = self._nodes
        for i in range(self._n_hidden + 1):
            hidden_layers.append(np.zeros((nodes[i + 1], 1)))
        return np.array(hidden_layers)
    
    def _forward_propagation(self):
        """
        Suppose 2 observations
        
        Suppose previous layer is 3 nodes
        Suppose current layer is 2 nodes
        
        prev shape (2,3)
        prev = ob1 [prev_node_1 val, prev_node_2 val, prev_node_3 val]
               ob2 [prev_node_1 val, prev_node_2 val, prev_node_3 val]
               
        layer shape (3,2)
        layer = [weight for current_node_1 for prev_node_1, weight for current_node_2 for prev_node_1]
                [weight for current_node_1 for prev_node_2, weight for current_node_2 for prev_node_2]
                [weight for current_node_1 for prev_node_3, weight for current_node_2 for prev_node_3]
                
        output shape (2,2) # since 2 observations and 2 layers
        output = ob1 [current_node_1 val, current_node_2 val]
                 ob2 [current_node_1 val, current_node_2 val]
                 
        Then for bias in current layer it is (2,1) since 2 nodes in current layer
        
        So for each row in output we add the bias row wise and apply the activation function to each row
        
        prev <- ouput
        
        Move onto next layer...
        """
        prev = self._X
        weights = self._weights
        biases = self._biases
        activations = self._activations[1:-1]
    
        for idx, layer in enumerate(weights):
            if idx == (len(weights) - 1):
                prev = (prev @ layer) + biases[idx].T,
            else:
                prev = self._activation(data = (prev @ layer) + biases[idx].T,
                                        activation = activations[idx])
        return prev
    
    def _train(self):
        # CURRENTLY 1 STEP OF FORWARD PROPGAGATION
        out = self._forward_propagation()
        print(out)
        
            
            
        
        

In [118]:
INPUT_SIZE = X_train_sample.shape[1]
OUTPUT_SIZE = 1
nodes = [INPUT_SIZE,4,3,OUTPUT_SIZE]
activations = ["relu" for i in range(len(nodes))]

nn = NeuralNetwork()

nn.fit(X = X_train_sample,
       y = X_train_sample,
       n_hidden = len(nodes) - 2,
       nodes = nodes,
       activations = activations)

(array([[-3.57730278e-04],
       [-3.98102084e-04],
       [-1.05427971e-04],
       [-5.70042831e-04],
       [-3.07911802e-05]]),)


In [86]:
nn._weights.shape

(3,)

In [88]:
nn._biases[2]

array([[0.]])